In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [18]:
pd.get_dummies(df)

0    68470
1      347
Name: loan_status_high_risk, dtype: int64

# Split the Data into Training and Testing

In [20]:
# Create our features
X = df.drop(columns="loan_status")
X = pd.get_dummies(X)
# Create our target
y = df['loan_status']

In [23]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [39]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification

#X_resampled, y_resampled = make_classification(n_samples=100, n_classes=2,
#                            n_informative=95, weights=[0.5, 0.5],
#                            random_state=1)

model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [40]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7877672625306695

In [41]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   58,    29],
       [ 1560, 15558]])

In [42]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.67      0.91      0.07      0.78      0.59        87
   low_risk       1.00      0.91      0.67      0.95      0.78      0.62     17118

avg / total       0.99      0.91      0.67      0.95      0.78      0.62     17205



In [64]:
# List the features sorted in descending order by feature importance
importance = model.feature_importances_
importance
# summarize feature importance
for i,v in enumerate(importance):
    for col in X_train.columns:
        print(col + ': %.5f' % (v))

loan_amnt: 0.00983
int_rate: 0.00983
installment: 0.00983
annual_inc: 0.00983
dti: 0.00983
delinq_2yrs: 0.00983
inq_last_6mths: 0.00983
open_acc: 0.00983
pub_rec: 0.00983
revol_bal: 0.00983
total_acc: 0.00983
out_prncp: 0.00983
out_prncp_inv: 0.00983
total_pymnt: 0.00983
total_pymnt_inv: 0.00983
total_rec_prncp: 0.00983
total_rec_int: 0.00983
total_rec_late_fee: 0.00983
recoveries: 0.00983
collection_recovery_fee: 0.00983
last_pymnt_amnt: 0.00983
collections_12_mths_ex_med: 0.00983
policy_code: 0.00983
acc_now_delinq: 0.00983
tot_coll_amt: 0.00983
tot_cur_bal: 0.00983
open_acc_6m: 0.00983
open_act_il: 0.00983
open_il_12m: 0.00983
open_il_24m: 0.00983
mths_since_rcnt_il: 0.00983
total_bal_il: 0.00983
il_util: 0.00983
open_rv_12m: 0.00983
open_rv_24m: 0.00983
max_bal_bc: 0.00983
all_util: 0.00983
total_rev_hi_lim: 0.00983
inq_fi: 0.00983
total_cu_tl: 0.00983
inq_last_12m: 0.00983
acc_open_past_24mths: 0.00983
avg_cur_bal: 0.00983
bc_open_to_buy: 0.00983
bc_util: 0.00983
chargeoff_within_

issue_d_Feb-2019: 0.00740
issue_d_Jan-2019: 0.00740
issue_d_Mar-2019: 0.00740
pymnt_plan_n: 0.00740
initial_list_status_f: 0.00740
initial_list_status_w: 0.00740
next_pymnt_d_Apr-2019: 0.00740
next_pymnt_d_May-2019: 0.00740
application_type_Individual: 0.00740
application_type_Joint App: 0.00740
hardship_flag_N: 0.00740
debt_settlement_flag_N: 0.00740
loan_amnt: 0.00795
int_rate: 0.00795
installment: 0.00795
annual_inc: 0.00795
dti: 0.00795
delinq_2yrs: 0.00795
inq_last_6mths: 0.00795
open_acc: 0.00795
pub_rec: 0.00795
revol_bal: 0.00795
total_acc: 0.00795
out_prncp: 0.00795
out_prncp_inv: 0.00795
total_pymnt: 0.00795
total_pymnt_inv: 0.00795
total_rec_prncp: 0.00795
total_rec_int: 0.00795
total_rec_late_fee: 0.00795
recoveries: 0.00795
collection_recovery_fee: 0.00795
last_pymnt_amnt: 0.00795
collections_12_mths_ex_med: 0.00795
policy_code: 0.00795
acc_now_delinq: 0.00795
tot_coll_amt: 0.00795
tot_cur_bal: 0.00795
open_acc_6m: 0.00795
open_act_il: 0.00795
open_il_12m: 0.00795
open_il_

mths_since_recent_inq: 0.00152
num_accts_ever_120_pd: 0.00152
num_actv_bc_tl: 0.00152
num_actv_rev_tl: 0.00152
num_bc_sats: 0.00152
num_bc_tl: 0.00152
num_il_tl: 0.00152
num_op_rev_tl: 0.00152
num_rev_accts: 0.00152
num_rev_tl_bal_gt_0: 0.00152
num_sats: 0.00152
num_tl_120dpd_2m: 0.00152
num_tl_30dpd: 0.00152
num_tl_90g_dpd_24m: 0.00152
num_tl_op_past_12m: 0.00152
pct_tl_nvr_dlq: 0.00152
percent_bc_gt_75: 0.00152
pub_rec_bankruptcies: 0.00152
tax_liens: 0.00152
tot_hi_cred_lim: 0.00152
total_bal_ex_mort: 0.00152
total_bc_limit: 0.00152
total_il_high_credit_limit: 0.00152
home_ownership_ANY: 0.00152
home_ownership_MORTGAGE: 0.00152
home_ownership_OWN: 0.00152
home_ownership_RENT: 0.00152
verification_status_Not Verified: 0.00152
verification_status_Source Verified: 0.00152
verification_status_Verified: 0.00152
issue_d_Feb-2019: 0.00152
issue_d_Jan-2019: 0.00152
issue_d_Mar-2019: 0.00152
pymnt_plan_n: 0.00152
initial_list_status_f: 0.00152
initial_list_status_w: 0.00152
next_pymnt_d_Apr-

### Easy Ensemble AdaBoost Classifier

In [65]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
model.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [66]:
# Calculated the balanced accuracy score
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.925427358175101

In [67]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   79,     8],
       [  979, 16139]])

In [68]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.91      0.94      0.14      0.93      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205

